# Examples from "Classical Approaches to Chiral Polaritonics"

## Import modules

In [ ]:
import chiral_transfermatrix as ct
import numpy as np
import matplotlib.pyplot as plt

## Dielectric permeability $\varepsilon$ and chiral coupling $\kappa$

In [ ]:
def eps_DL(omega, epsinf, omegap, omega0=0, gamma=0, kappa0=0):
    """Drude-Lorentz model for the dielectric function of a material."""
    eps = epsinf + omegap**2 / (omega0**2 - omega**2 - 1j * gamma * omega)
    # the +1e-200 prevents division by zero when omega0=0 (which is only valid if kappa0=0)
    kappa = kappa0 * omega / (omega0+1e-200) * (eps-epsinf)
    return eps, kappa

## Fig. 9a

In [ ]:
# use numpy broadcasting rules to scan over d and omega, results will have indices [i_omega, i_d]
d = np.linspace(10, 500, 500)
omega = np.linspace(1.6, 2.4, 500)[:,None] # in eV
lambda_vac = 1239.8419843320028 / omega # lambda in nm, "magic" constant is hc in eV*nm

eps_Ag,  k_Ag  = eps_DL(omega, epsinf=4.77574276, omegap=9.48300763, omega0=0,   gamma=0.17486845, kappa0=0.0)
eps_mol, k_mol = eps_DL(omega, epsinf=2.89,       omegap=0.05,       omega0=2.0, gamma=0.05,       kappa0=1e-3)

air_infty = ct.MaterialLayer(d=np.inf, eps=1)
Ag_mirror = ct.MaterialLayer(d=30,     eps=eps_Ag)
molecules = ct.MaterialLayer(d=d,      eps=eps_mol, kappa=k_mol)

mls_Ag = ct.MultiLayerScatt([air_infty, Ag_mirror, molecules, Ag_mirror, air_infty], lambda_vac, theta0=0.)

### Plot

In [ ]:
plt.pcolormesh(d, omega, mls_Ag.Tsp, cmap = "turbo", shading = "gouraud")
plt.axhline(y=2.0, color='w', linestyle='--', linewidth=2)
cbar = plt.colorbar()
plt.xlabel(r"$L$ (nm)")
plt.ylabel(r"$\hbar\omega$ (eV)")
cbar.set_label(r"$T_{\perp,+}$");

## Fig. 14a

In [ ]:
d = np.linspace(70, 450, 500)
omega  = np.linspace(1.5, 2.9, 500)[:,None]
lambda_vac = 1239.8419843320028 / omega # lambda in nm, "magic" constant is hc in eV*nm

eps_mol, k_mol = eps_DL(omega=omega, epsinf=2.89, omegap=0.5, omega0=2.2, gamma=0.05, kappa0=1e-3)

air_infty = ct.MaterialLayer(d=np.inf, eps=1)
air_thin  = ct.MaterialLayer(d=0,      eps=1)
molecules = ct.MaterialLayer(d=d,      eps=eps_mol, kappa=k_mol)

omegaPR = 2
gammaPR = 0.01
mirror_1 = ct.helicity_preserving_mirror(omega,omegaPR=omegaPR,gammaPR=gammaPR,enantiomer=False)
mirror_2 = ct.helicity_preserving_mirror(omega,omegaPR=omegaPR,gammaPR=gammaPR,enantiomer=True)

mls_HP = ct.MultiLayerScatt([air_infty, mirror_1, air_thin, molecules, air_thin, mirror_2, air_infty], lambda_vac, theta0=0.)

### Plot

In [ ]:
plt.pcolormesh(d, omega, mls_HP.Tsp, cmap="turbo", shading="gouraud")
cbar = plt.colorbar()
plt.xlabel(r"$L$ (nm)")
plt.ylabel(r"$\hbar\omega$ (eV)")
cbar.set_label(r"$T_{\perp,+}$")